In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

## UpstageLayoutAnalysisLoader


표와 그림을 포함한 모든 문서에서 문서 요소를 감지합니다.

먼저, 소프트웨어정책연구소(SPRi) - 2023년 12월호 문서를 로드하고 분절하는 예제를 살펴보겠습니다.

- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`


In [ ]:
# from RAGchain.preprocess.loader import Win32HwpLoader
from RAGchain.preprocess.loader import RustHwpLoader

loader = RustHwpLoader("./data/(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwp")

# loader = Win32HwpLoader("./data/(첨부5) 2024년도 하반기 중남미 지역기구 파견인턴 선발 공고.hwpx")
documents = loader.load()

len(documents)

In [ ]:
from pprint import pprint
pprint(documents[0].page_content)

In [ ]:
# 첫번째 문서의 내용 출력
print(split_docs[0].page_content)
# 첫번째 문서의 메타데이터 출력
print(split_docs[0].metadata)

In [ ]:
split_docs[3]

다음은, 신한라이프 사업방법서로 이미지형 PDF 문서를 로드하고 분절하는 예제를 살펴보겠습니다.

- 파일명: `사업방법서_신한(간편가입)홈닥터의료비보장보험(무배당, 갱신형)_20240401_P14.pdf`



In [ ]:
# 대상 이미지 PDF 파일 로드. 파일의 경로와 분할 방식 입력
loader = UpstageLayoutAnalysisLoader("data/사업방법서_신한(간편가입)홈닥터의료비보장보험(무배당, 갱신형)_20240401_P14.pdf", split="page")

# 페이지 별 문서 로드
split_docs = loader.load()  # or loader.lazy_load()

print(f"문서의 수: {len(split_docs)}")

In [ ]:
for doc in split_docs[:3]:
    print(doc)

이번에는, 신한라이프 사업방법서로 텍스트형 PDF 문서를 로드하고 분절하는 예제를 살펴보겠습니다.

- 파일명: `사업방법서_(간편)신한통합건강보장보험원(ONE)(무배당, 해약환급금 미지급형)_20240503_v0.1_P34.pdf`


In [ ]:
# 신한라이프 사업방법서로 텍스트형 PDF 문서를 로드

loader = UpstageLayoutAnalysisLoader("data/사업방법서_(간편)신한통합건강보장보험원(ONE)(무배당, 해약환급금 미지급형)_20240503_v0.1_P34.pdf", split="page")
docs = loader.load()
print(f"문서의 수: {len(docs)}")

In [ ]:
docs[0].metadata

In [ ]:
docs[0].page_content

In [ ]:
from bs4 import BeautifulSoup

def extract_text(html_content):
    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(html_content, 'html.parser')

    # 모든 태그의 텍스트 추출 및 공백으로 연결
    text_content = ' '.join(soup.stripped_strings)
    return text_content

# 함수 사용 예
html_content = docs[0].page_content
extracted_text = extract_text(html_content)
print(extracted_text)

In [ ]:
split_docs[3].page_content

In [ ]:
extracted_text = extract_text(docs[3].page_content)
print(extracted_text)

## 문서 로드 시간 비교

신한라이프 사업방법서로 이미지형 PDF 문서를 로드하여 각각의 소요시간을 비교해 봅니다.

- 파일명: `사업방법서_신한(간편가입)홈닥터의료비보장보험(무배당, 갱신형)_20240401_P14.pdf`

In [ ]:
# UpstageLayoutAnalysisLoader 와 PyPDFLoader의 로드 시간 비교
from langchain.document_loaders import PyPDFLoader
from langchain_upstage import UpstageLayoutAnalysisLoader
import time

# PyPDFLoader 로드 시간 측정
pypdf_loader = PyPDFLoader("data/사업방법서_신한(간편가입)홈닥터의료비보장보험(무배당, 갱신형)_20240401_P14.pdf")
start_time = time.time()
split_docs = pypdf_loader.load_and_split()
end_time = time.time()
pypdf_duration = end_time - start_time
print(f"PyPDFLoader 문서의 수: {len(split_docs)}, 로드 시간: {pypdf_duration:.2f}초")

# UpstageLayoutAnalysisLoader 로드 시간 측정
upstage_loader = UpstageLayoutAnalysisLoader("data/사업방법서_신한(간편가입)홈닥터의료비보장보험(무배당, 갱신형)_20240401_P14.pdf", split="page")
start_time = time.time()
docs = upstage_loader.load()
end_time = time.time()
upstage_duration = end_time - start_time
print(f"UpstageLayoutAnalysisLoader 문서의 수: {len(docs)}, 로드 시간: {upstage_duration:.2f}초")

# 시간 비교 출력
# print(f"로드 시간 비교: PyPDFLoader - {pypdf_duration:.2f}초, UpstageLayoutAnalysisLoader - {upstage_duration:.2f}초")
if pypdf_duration < upstage_duration:
    slower_by = ((upstage_duration - pypdf_duration) / pypdf_duration) * 100
    print(f"UpstageLayoutAnalysisLoader는 PyPDFLoader보다 {slower_by:.2f}% 느립니다.")
else:
    slower_by = ((pypdf_duration - upstage_duration) / upstage_duration) * 100
    print(f"PyPDFLoader는 UpstageLayoutAnalysisLoader보다 {slower_by:.2f}% 느립니다.")

**그러나 UpstageLayoutAnalysisLoader는 PyPDFLoader가 못가지고오는 이미지 PDF의 텍스트를 가져옴**

In [ ]:
# PyPDFLoader의 텍스트 추출 예, page_content가 비었고 메타데이터만 출력됨
split_docs[0]


In [ ]:
# UpstageLayoutAnalysisLoader의 텍스트 추출 예, page_content가 출력됨
docs[0]